In this notebook, we will generate poems in the style of Robert Frost using a 2nd order Markov model. This means that the next token depends on the previous *two* tokens. 

One important difference wrt 'markov_text_classifier.ipynb' is that here we will use dictionaries to store the transition matrices A_ijk, rather than lists. The reason for this is computational efficiency. As we increase the order of the Markov model to N, the size of the A_{...} matrix grows as V^(N+1), where V - vocabulary size. However, because of the curse of dimensionality, the volume of data is increasingly low relative to the full volume of the matrix - that is, the matrix is increasingly sparse. Storing only the actual non-zero values in a dictionary is more efficient. We will need a nested dict - in this case, it will be nested twice, e.g. 

A['my']['cute']['dog'] = 0.3

----------
Another thing that is different here is that we will not be doing classification, so we won't be calculating probabilities of provided sequences. Rather, we will be sampling random tokens according to their probability distributions encoded in the nested dict. For example, if:

A['my'] = {'cute': 0.2, 'own': 0.5, 'son': 0.3}

then we need to randomly sample a float between $x \in [0.0, 1.0]$ and if $x<0.2$, then we sample 'cute', if $0.2<x<0.7$, then we sample 'own', and so on.

----------
The final note is that we still need the first-order transition matrix A_ij, but it should not be trained on all the transitions - it needs to be trained only on the transitions from the first token to the second one. After that, the second-order transition matrix is used.

In [ ]:
import numpy as np
import pandas as pd

import string
# from sklearn.model_selection import train_test_split
from nltk import word_tokenize

We will generate poems in the style of Robert Frost by training a second-order Markov model on his poems and then sampling tokens by randomly drawing numbers from the uniform distribution $[0, 1]$.

In [ ]:
!wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/robert_frost.txt

In [ ]:
input_files = [
  'edgar_allan_poe.txt',
  'robert_frost.txt',
]

In [ ]:
# collect data into lists
input_text = []

for line in open('robert_frost.txt'):
  line = line.rstrip().lower()
  if line:
    # remove punctuation
    line = line.translate(str.maketrans('', '', string.punctuation))
    input_text.append(line)

Convert lines to tokens:

In [ ]:
vocab = []
X_train = []

for line in input_text:
    tokenised_line = word_tokenize(line)
    X_train.append(tokenised_line)
    for tok in tokenised_line:
        if tok not in vocab:
            vocab.append(tok)

V = len(vocab)
print(f'Vocab length: {V}')

We don't convert tokens to indices because we will store our data in nested dictionaries instead.

In [ ]:
pi_i = {}
A_ij = {}
A_ijk = {}

N = len(X_train)

Function to add a key to dictionary or extend the value if the key already exists:

In [ ]:
def add_to_dict(dictionary, key, val):
    if key not in dictionary.keys():
        dictionary[key] = []
    dictionary[key].append(val)

In [ ]:
for line in X_train:
    for ii in range(len(line)): # loop through every token one-by-one
        tok = line[ii]

        if ii==0:
            pi_i[tok] = pi_i.get(tok, 0) + 1
        else:
            tok_1 = line[ii-1]
            
            # if we're at the last to token, append and 'END' flag
            # note that this line is crucial
            # otherwise we'll reach tokens that were at the end of a line,
            # but the program will want to continue generating the next tokens instead of terminating the line
            if ii == len(line)-1:
                add_to_dict(A_ijk, (tok_1, tok), 'END')

            if ii==1:
                add_to_dict(A_ij, tok_1, tok)
            else:
                tok_2 = line[ii-2]
                add_to_dict(A_ijk, (tok_2, tok_1), tok)


Now convert the collected lists of next tokens into the probability for next tokens. This is done simply by counting the total number of possible tokens that could follow a previous token and dividing by this number.

In [ ]:
pi_i.values()
for key, val in pi_i.items():
    pi_i[key] = val / N

In [ ]:
def list_to_prob(tokens: list[str]):
    n = len(tokens)
    new_dict = {}

    # append counts for distinct tokens in the list [tok1, tok2, tok3, tok1, tok3, ...]
    for tok in tokens:
        new_dict[tok] = new_dict.get(tok, 0) + 1
    
    # normalise by the length of that list
    for tok, val in new_dict.items():
        new_dict[tok] = val / n

    return new_dict

In [ ]:
for key, val in A_ij.items():
    A_ij[key] = list_to_prob(val)

for key, val in A_ijk.items():
    A_ijk[key] = list_to_prob(val)

At this point, we have the first- and second-order transition values, as well as the initial state probabilities. We now want to draw a random token from these dictionaries in order to generate text.

In [ ]:
def draw_random_token(d):
    random_no = np.random.uniform()

    prob_sum = 0
    for key in d.keys():
        prob_sum += d[key]
        if prob_sum > random_no:
            break
    return key

In [ ]:
def Markov_poem_generator(n_lines, pi_i, A_ij, A_ijk):
    poem = []

    for _ in range(n_lines):
        verse = []
        first_token = draw_random_token(pi_i)
        verse.append(first_token)
        second_token = draw_random_token(A_ij[first_token])
        verse.append(second_token)

        while True:
            tok = draw_random_token(A_ijk[(first_token, second_token)])
            # print(tok)
            if tok == 'END':
                break
            verse.append(tok)
            first_token = second_token
            second_token = tok
        poem.append(' '.join(verse))
    
    return poem

In [ ]:
Markov_poem_generator(5, pi_i, A_ij, A_ijk)

We can compare the number of items stored in the dictionaries to the number of items we would have to store in sparse matrices:

In [ ]:
length_pi_i = len(pi_i)
length_A_ij = len(A_ij)
length_A_ijk = len(A_ijk)

for value in A_ij.values():
    length_A_ij += len(value)

for value in A_ijk.values():
    length_A_ijk += len(value)

print(f'Number of items stored in pi_i dict / number of items stored in pi_i matrix: {length_pi_i / V}')
print(f'Number of items stored in A_ij dict / number of items stored in A_ij matrix: {length_A_ij / (V*V)}')
print(f'Number of items stored in A_ijk dict / number of items stored in A_ijk matrix: {length_A_ijk / (V*V*V)}')

We can clearly see the 'curse of dimensionality' here.